In [1]:
import pyspark
import math
from pyspark.mllib.recommendation import ALS, MatrixFactorizationModel, Rating
from settings import *


# Model Evaluation Using 10M Data

In [3]:
# Sorting data by time
rdd =  sc.textFile(RATINGS_10M).map(lambda line: [float(x) for x in line.split('::')]).sortBy(lambda x: x[3],False)

In [66]:
rdd.first()

[62510.0, 34148.0, 3.0, 1231131736.0]

In [4]:
size = rdd.count()

In [5]:
# Processing data to structure: Rating(user=62510, product=34148, rating=3.0)
train = rdd.zipWithIndex().filter(lambda x: x[-1] < size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))

In [7]:
print train.first()

Rating(user=62510, product=34148, rating=3.0)


In [9]:
# Processing validation data
testEval = rdd.zipWithIndex().filter(lambda x: x[-1] > size*0.6 and x[-1] < size*0.8).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))
test = testEval.map(lambda p: (p[0], p[1]))

In [12]:
# Training ALS Model
latentFactors = [10,20,30,40,50]
numIterations = 25
regParameter = [0.01,0.1,1.0,10.0]

for i in range(5):
    for j in range(4):
        model = ALS.train(train, latentFactors[i], numIterations, regParameter[j])
        predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
        ratesAndPreds = testEval.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
        RMSE = ratesAndPreds.map(lambda r: math.sqrt((r[1][0] - r[1][1])**2)).mean()
        print("No. of latent factors: " + str(latentFactors[i]) + "\t Regulation Parameter: " + str(regParameter[j]))
        print("RMSE = " + str(RMSE))
        print "\n"
    

No. of latent factors: 10	 Regulation Parameter: 0.01
RMSE = 0.994771720874


No. of latent factors: 10	 Regulation Parameter: 0.1
RMSE = 0.832598364153


No. of latent factors: 10	 Regulation Parameter: 1.0
RMSE = 1.97907550365


No. of latent factors: 10	 Regulation Parameter: 10.0
RMSE = 13.644303884


No. of latent factors: 20	 Regulation Parameter: 0.01
RMSE = 1.06994780361


No. of latent factors: 20	 Regulation Parameter: 0.1
RMSE = 0.834236910709


No. of latent factors: 20	 Regulation Parameter: 1.0
RMSE = 1.97907550525


No. of latent factors: 20	 Regulation Parameter: 10.0
RMSE = 13.644303884


No. of latent factors: 30	 Regulation Parameter: 0.01
RMSE = 1.1039384923


No. of latent factors: 30	 Regulation Parameter: 0.1
RMSE = 0.834441417826


No. of latent factors: 30	 Regulation Parameter: 1.0
RMSE = 1.97907550497


No. of latent factors: 30	 Regulation Parameter: 10.0
RMSE = 13.644303884


No. of latent factors: 40	 Regulation Parameter: 0.01
RMSE = 1.13017770616


No. o

# Testing Data Using 10M Data

In [10]:
# Processing test data
testData = rdd.zipWithIndex().filter(lambda x: x[-1] > size*0.8).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))
test = testData.map(lambda p: (p[0], p[1]))

In [13]:
# Training Best ALS Model
latentFactors = 10
numIterations = 25
regParameter = 0.1

model = ALS.train(train, latentFactors, numIterations, regParameter)
predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testData.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
RMSE = ratesAndPreds.map(lambda r: math.sqrt((r[1][0] - r[1][1])**2)).mean()
print("No. of latent factors: " + str(latentFactors) + "\t Regulation Parameter: " + str(regParameter))
print("RMSE = " + str(RMSE))


Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 1 in stage 1284.0 failed 1 times, most recent failure: Lost task 1.0 in stage 1284.0 (TID 1679, localhost): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/chongyeegan/spark-1.6.1/python/lib/pyspark.zip/pyspark/worker.py", line 111, in main
    process()
  File "/Users/chongyeegan/spark-1.6.1/python/lib/pyspark.zip/pyspark/worker.py", line 106, in process
    serializer.dump_stream(func(split_index, iterator), outfile)
  File "/Users/chongyeegan/spark-1.6.1/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/chongyeegan/spark-1.6.1/python/pyspark/rdd.py", line 2346, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/Users/chongyeegan/spark-1.6.1/python/pyspark/rdd.py", line 

# Recommendation System Using 10M Data

In [7]:
testUser = train.first()[0]

In [8]:
traindict = train.map(lambda data: (data[0], [data[1]])).reduceByKey(lambda a,b: a+b).collectAsMap()

In [ ]:
## Uncomment to parse a ratings file customratings.dat instead
#rdd2 = sc.textFile(RATINGS_CUSTOM).map(lambda line: [float(x) for x in line.split('::')])
#rdd2 = rdd2.map(lambda x: Rating(int(x[0]), int(x[1]), x[2]))

#train = train.union(rdd2)
#traindict = train.map(lambda data: (data[0], [data[1]])).reduceByKey(lambda a,b: a+b).collectAsMap()

#testUser = rdd2.map(lambda x: int(x[0]))

In [9]:
# Creat recommended movies that user have not watched before

topMovies = model.recommendProducts(testUser, 100)
filteredMovies = []
for row in topMovies:
    if not row[1] in traindict[row[0]]:
        filteredMovies.append(row)


In [ ]:
for i in range(5):
    print filteredMovies[i]

In [65]:
# Print out filtered Movies as string (names)
movies10M_dict =  sc.textFile(MOVIES_10M).map(lambda line: line.split('::')).map(lambda x: (float(x[0]), x[1])).collectAsMap()

for i in range(5):
    print movies10M_dict[filteredMovies[i][1]]


Shadows of Forgotten Ancestors (1964)
Satan's Tango (Sátántangó) (1994)
Man Who Planted Trees, The (Homme qui plantait des arbres, L') (1987)
More (1998)
Maradona by Kusturica (2008)


# Model Evaluation Using 22M

In [2]:
rdd =  sc.textFile(RATINGS_22M)
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)
rdd = rdd.map(lambda line: [float(x) for x in line.split(',')]).sortBy(lambda x: x[3],False)

In [3]:
size = rdd.count()

In [4]:
train = rdd.zipWithIndex().filter(lambda x: x[-1] < size*0.6).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))

In [5]:
# Training ALS Model
latentFactors = 10
numIterations = 25
regParameter = 0.1

model = ALS.train(train, latentFactors, numIterations, regParameter)

In [6]:
# Processing test data
testData = rdd.zipWithIndex().filter(lambda x: x[-1] > size*0.8).map(lambda x: Rating(int(x[0][0]), int(x[0][1]), x[0][2]))
test = testData.map(lambda p: (p[0], p[1]))

In [7]:
predictions = model.predictAll(test).map(lambda r: ((r[0], r[1]), r[2]))
ratesAndPreds = testData.map(lambda r: ((r[0], r[1]), r[2])).join(predictions)
RMSE = ratesAndPreds.map(lambda r: math.sqrt((r[1][0] - r[1][1])**2)).mean()
print("No. of latent factors: " + str(latentFactors) + "\t Regulation Parameter: " + str(regParameter))
print("RMSE = " + str(RMSE))

No. of latent factors: 10	 Regulation Parameter: 0.1
RMSE = 0.738841244577


# Recommendation System - 22M Data

In [8]:
testUser = train.first()[0]

In [9]:
traindict = train.map(lambda data: (data[0], [data[1]])).reduceByKey(lambda a,b: a+b).collectAsMap()

In [10]:
topMovies = model.recommendProducts(testUser, 100)
filteredMovies = []
for row in topMovies:
    if not row[1] in traindict[row[0]]:
        filteredMovies.append(row)
#filteredMovies = topMovies.filter(lambda x: not x[1] in traindict[x[0]])


In [11]:
for i in range(5):
    print filteredMovies[i]

Rating(user=73725, product=142891, rating=6.0114232865355515)
Rating(user=73725, product=150900, rating=5.855408786545276)
Rating(user=73725, product=144202, rating=5.84263406216658)
Rating(user=73725, product=149268, rating=5.824363774857004)
Rating(user=73725, product=148857, rating=5.824363774857004)


In [30]:
# Print out filtered Movies as string (names)
moviesrdd =  sc.textFile(MOVIES_22M)
header = moviesrdd.first()
moviesrdd = moviesrdd.filter(lambda line: line != header)

movies22M_dict =  moviesrdd.map(lambda line: line.split(',')).map(lambda x: (int(x[0]), x[1])).collectAsMap()

for i in range(5):
    print movies22M_dict[filteredMovies[i][1]]

The Legend of Paul and Paula (1973)
Top Gear: The Worst Car In the History of the World (2012)
Catch That Girl (2002)
The Pied Piper (1942)
"Christmas
